# **ROLL-RATE ANALYSIS**

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import utils.utils as utils

# !pip freeze > ../requirements.txt

In [8]:
data = pd.read_csv(r'..\data\final_data.csv').drop(['Unnamed: 0'], axis=1)